In [1]:
# Test all combinations of knockouts
import pandas as pd
import cobra
from collections import OrderedDict
import copy
from copy import deepcopy
import os
from itertools import combinations

In [2]:
curr_dir = os.getcwd()
# Metabolic model (COBRApy json)
model = cobra.io.load_json_model(curr_dir + '/../iRhtoCN.json')
biomass = 'BIOMASS'

In [9]:
# Define the list of reactions to knock out
# all_reactions = ["reaction1_id", "reaction2_id", "reaction3_id"]
all_reactions = set(['ACGAM6S1_r','ACGAM6S2_r','DPGM_c','RPE_c','TKT1_c','TKT2_c'])
#all_reactions = set(['CHTNDA_c','RPE_c','SDPDS_c'])

In [10]:
# Function to perform FBA and categorize reactions
def categorize_reactions(model, reactions_to_knockout, essential_reactions, feasible_knockouts):
    # Create a copy of the model to avoid modifying the original model
    model_copy = model.copy()

    # Knock out the reactions in the current combination
    for reaction_id in reactions_to_knockout:
        reaction = model_copy.reactions.get_by_id(reaction_id)
        reaction.lower_bound = 0.0
        reaction.upper_bound = 0.0

    # Perform FBA on the modified model
    model_copy.objective = model.reactions.get_by_id(biomass)
    model_copy.objective_direction = "max"
    knockout_solution = model_copy.optimize()
    # knockout_solution = cobra.flux_analysis.optimize_minimal_flux(model_copy)

    # Check the objective value
    if knockout_solution.objective_value <= 0:
        essential_reactions.update(reactions_to_knockout)
        print("Lethal knockout: ",reactions_to_knockout)
    else:
        feasible_knockouts.append(reactions_to_knockout)
        print("Nonlethal knockout: ",reactions_to_knockout)

In [11]:
# Lists to store results
essential_reactions = set()
feasible_single_knockouts = []
feasible_knockout_combinations = [feasible_single_knockouts]

In [12]:
# Try single knockouts
for reaction_id in all_reactions:
    categorize_reactions(model, [reaction_id], essential_reactions, feasible_single_knockouts)
# Print the results
print("Essential Reactions:")
print(essential_reactions)
print("\nFeasible Single Knockouts:")
print(feasible_single_knockouts)

Nonlethal knockout:  ['RPE_c']
Nonlethal knockout:  ['ACGAM6S2_r']
Nonlethal knockout:  ['TKT1_c']
Nonlethal knockout:  ['DPGM_c']
Nonlethal knockout:  ['ACGAM6S1_r']
Nonlethal knockout:  ['TKT2_c']
Essential Reactions:
set()

Feasible Single Knockouts:
[['RPE_c'], ['ACGAM6S2_r'], ['TKT1_c'], ['DPGM_c'], ['ACGAM6S1_r'], ['TKT2_c']]


In [13]:
# Function to generate higher-order knockout combinations
def generate_higher_order_combinations(feasible_knockouts, n):
    higher_order_combinations = []
    for combo in combinations(feasible_knockouts, n):
        combined_knockout = set(item for sublist in combo for item in sublist)
        higher_order_combinations.append(combined_knockout)
    return higher_order_combinations

In [14]:
# feasible_double_knockouts = []
# # Try double knockouts
# for combo in combinations(all_reactions, 2):
#     combined_knockout = list(combo)
#     categorize_reactions(model, combined_knockout, essential_reactions, feasible_double_knockouts)

# Try higher-order knockouts (e.g., double, triple, quadruple)
for order in range(2, len(all_reactions)):
    higher_order_combinations = generate_higher_order_combinations(feasible_knockout_combinations[-1], order)
    feasible_knockout_combinations.append([])
    for combo in higher_order_combinations:
        categorize_reactions(model, combo, essential_reactions, feasible_knockout_combinations[-1])

Nonlethal knockout:  {'RPE_c', 'ACGAM6S2_r'}
Nonlethal knockout:  {'RPE_c', 'TKT1_c'}
Nonlethal knockout:  {'RPE_c', 'DPGM_c'}
Nonlethal knockout:  {'RPE_c', 'ACGAM6S1_r'}
Lethal knockout:  {'RPE_c', 'TKT2_c'}
Nonlethal knockout:  {'ACGAM6S2_r', 'TKT1_c'}
Nonlethal knockout:  {'ACGAM6S2_r', 'DPGM_c'}
Nonlethal knockout:  {'ACGAM6S2_r', 'ACGAM6S1_r'}
Nonlethal knockout:  {'ACGAM6S2_r', 'TKT2_c'}
Nonlethal knockout:  {'DPGM_c', 'TKT1_c'}
Nonlethal knockout:  {'ACGAM6S1_r', 'TKT1_c'}
Lethal knockout:  {'TKT1_c', 'TKT2_c'}
Nonlethal knockout:  {'ACGAM6S1_r', 'DPGM_c'}
Nonlethal knockout:  {'DPGM_c', 'TKT2_c'}
Nonlethal knockout:  {'ACGAM6S1_r', 'TKT2_c'}
Nonlethal knockout:  {'RPE_c', 'DPGM_c', 'ACGAM6S2_r', 'TKT1_c'}
Nonlethal knockout:  {'RPE_c', 'ACGAM6S1_r', 'ACGAM6S2_r', 'TKT1_c'}
Nonlethal knockout:  {'RPE_c', 'ACGAM6S2_r', 'TKT1_c'}
Nonlethal knockout:  {'RPE_c', 'DPGM_c', 'ACGAM6S2_r', 'TKT1_c'}
Nonlethal knockout:  {'RPE_c', 'ACGAM6S1_r', 'ACGAM6S2_r', 'TKT1_c'}
Lethal knockout:  

KeyboardInterrupt: 

In [ ]:
print("Essential Reactions:")
print(essential_reactions)
print("\nFeasible Single Knockouts:")
print(feasible_single_knockouts)
print("\nFeasible Knockout Combinations:")
for i, combinations in enumerate(feasible_knockout_combinations):
    print(f"Order-{i + 2} Knockouts:")
    print(combinations)